In [1]:
import tensorflow as tf

In [8]:
x = tf.constant(5.0)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x**3

In [9]:
y

<tf.Tensor: id=34, shape=(), dtype=float32, numpy=125.0>

In [10]:
tape.gradient(y, x)

<tf.Tensor: id=40, shape=(), dtype=float32, numpy=75.0>

In [19]:
x = tf.Variable(6.0, trainable=True)
with tf.GradientTape() as tape:
    y = x**3
tape.gradient(y, x)

<tf.Tensor: id=168, shape=(), dtype=float32, numpy=108.0>

In [24]:
x = tf.Variable(6.0)
with tf.GradientTape() as tape:
    y = x**3
tape.gradient(y, x)

<tf.Tensor: id=266, shape=(), dtype=float32, numpy=108.0>

In [11]:
x = tf.Variable(3.0, trainable=True)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    y = x**3

print(tape.gradient(y, x)) # -> None

None


In [17]:
x = tf.Variable(3.0, trainable=True)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    y = x**3

print(tape.gradient(y, x)) # -> None

tf.Tensor(27.0, shape=(), dtype=float32)


In [20]:
# Cannot rerun (not set as persistent)
tape.gradient(y, x)

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

In [21]:
# Higher-Order Derivatives
x = tf.Variable(3.0, trainable=True)
with tf.GradientTape() as tape1:
    with tf.GradientTape() as tape2:
        y = x ** 3
    order_1 = tape2.gradient(y, x)
order_2 = tape1.gradient(order_1, x)

print(order_2.numpy()) # -> 18.0

18.0


In [22]:
a = tf.Variable(6.0, trainable=True)
b = tf.Variable(2.0, trainable=True)
with tf.GradientTape(persistent=True) as tape:
    y1 = a ** 2
    y2 = b ** 3
                                                                                                                                                                                                                                                                                                                                                
print(tape.gradient(y1, a).numpy())
print(tape.gradient(y2, b).numpy())

12.0
12.0


In [23]:
print(tape.gradient(y1, a).numpy())
print(tape.gradient(y2, b).numpy())

12.0
12.0


In [30]:
x = tf.Variable(6.0)
with tf.GradientTape(persistent=True) as tape:
    y = x**3
tape.gradient(y, x)

<tf.Tensor: id=358, shape=(), dtype=float32, numpy=108.0>

In [33]:
 
tape.gradient(y, x)

<tf.Tensor: id=376, shape=(), dtype=float32, numpy=108.0>

In [34]:
x.assign(7)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=7.0>

In [36]:
tape.gradient(y, x)

<tf.Tensor: id=397, shape=(), dtype=float32, numpy=108.0>

In [38]:
x = tf.Variable(4.0, trainable=True)
with tf.GradientTape() as tape:
    y = x**3
    with tape.stop_recording():
        print(tape.gradient(y, x).numpy()) # -> 27.0

48.0


In [45]:
a = tf.Variable(6.0, trainable=True)
b = tf.Variable(2.0, trainable=True)
c = tf.Variable(3.0, trainable=True)
with tf.GradientTape(persistent=True) as tape:
    y1 = a ** 2
    with tape.stop_recording():
        print(tape.gradient(y1, a).numpy())
    
    y2 = b ** 3
    with tape.stop_recording():
        print(tape.gradient(y2, b).numpy())
    y3 = 2 * c
    with tape.stop_recording():
        print(tape.gradient(y3, c).numpy())
    print(tape.watched_variables())

12.0
12.0
2.0
(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>)


Linear Regression

In [46]:
import numpy as np
import random

In [47]:
# Loss function
def loss(real_y, pred_y):
    return tf.abs(real_y - pred_y)

In [48]:
# Training data
x_train = np.asarray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y_train = np.asarray([i*10+5 for i in x_train]) # y = 10x+5

In [57]:
# Trainable variables
a = tf.Variable(random.random(), trainable=True)
b = tf.Variable(random.random(), trainable=True)
a,b

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.6450149>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.98859453>)

In [54]:
a,b

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.67666644>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.25877672>)

In [59]:
def step(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        # Make prediction
        pred_y = a * real_x + b
        # Calculate loss
        reg_loss = loss(real_y, pred_y)
    
    # Calculate gradients
    a_gradients, b_gradients = tape.gradient(reg_loss, (a, b))

    # Update variables
    a.assign_sub(a_gradients * 0.001)
    b.assign_sub(b_gradients * 0.001)

In [60]:
for _ in range(100000):
    step(x_train, y_train)

In [61]:
print(f'y ≈ {a.numpy()}x + {b.numpy()}')

y ≈ 9.999174118041992x + 4.9906325340271


Polynomial regression
Exemple: y = a * x**2 + b * x + c

In [74]:
x_train = np.arange(10.0)
x_train

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [76]:
y_train = 8 * x_train **2 + 6 * x_train + 2
y_train

array([  2.,  16.,  46.,  92., 154., 232., 326., 436., 562., 704.])

In [71]:
# Init variables : random
a = tf.Variable(np.random(), trainable=True)
b = tf.Variable(np.random(), trainable=True)
c = tf.Variable(np.random(), trainable=True)

In [79]:
# step2 using Gradient Tap
learning_rate = 0.001
def step2(x_real, y_real):
    with tf.GradientTape(persistent=True) as tape:
        # calculate prediction
        y_predicted = a * x_real**2 + b * x_real + c
        # get loss 
        loss_value = loss(y_real, y_predicted)

        # Get loss function gradients with respect to a, b and c
    loss_gradient_a, loss_gradient_b, loss_gradient_c = tape.gradient(loss_value, (a, b, c))

    # Next parameters values
    a.assign_sub(loss_gradient_a * learning_rate)
    b.assign_sub(loss_gradient_b * learning_rate)
    c.assign_sub(loss_gradient_c * learning_rate)



In [94]:
# Training 
for _ in range(20000):
    step2(x_train, y_train)

In [93]:
print(f'y = {a.numpy()} x**2 + {b.numpy()} x + {c.numpy()}')

y = 7.996129035949707 x**2 + 5.667584419250488 x + 1.9999451637268066
